### Импорт данных для обучения

In [7]:
import pandas as pd
features = pd.read_csv('../../data/features.csv', index_col='match_id')

### Подготовка признаков для обучения

#### Удаление признаков, связанных с итогами матча 

In [8]:
final_feature_names = ['duration',
                       'radiant_win',
                       'tower_status_radiant',
                       'tower_status_dire',
                       'barracks_status_radiant',
                       'barracks_status_dire']

In [14]:
x = features.copy()
x = x.drop(final_feature_names, 1)
print (x.shape)

(97230, 102)


#### Обработка пропусков

##### Поиск пропусков

In [17]:
feature_names = list(x.columns.values)
object_count, feature_count = x.shape

In [18]:
for i, nonempty_count in enumerate(x.count()):
    feature_name = feature_names[i]
    empty_count = object_count - nonempty_count
    if empty_count > 0:
        print("%s: %s" % (feature_name, empty_count))

first_blood_time: 19553
first_blood_team: 19553
first_blood_player1: 19553
first_blood_player2: 43987
radiant_bottle_time: 15691
radiant_courier_time: 692
radiant_flying_courier_time: 27479
radiant_first_ward_time: 1836
dire_bottle_time: 16143
dire_courier_time: 676
dire_flying_courier_time: 26098
dire_first_ward_time: 1826


##### Объяснение возможных причин пропусков

* `first_blood_time` / `first_blood_team` / `first_blood_player1` - в течение первых пяти минут игроков не атаковали. При этом `first_blood_player2` больше, чем `first_blood_player1`, видимо, из-за того, что иногда первая кровь проливается в результате атак неигровых персонажей.
* Остальные пропуски говорят о том, что предметы, соответствующие колонкам, не использовались в течение первых пяти минут.

##### Заполнение пропусков

In [19]:
x = x.fillna(0)

#### Подготовка целевых переменных

Целевой признак -- `radiant_win`. Он уже представляет собой результат принадлежности к классам 1 (победила команда Radiant) и 0.

In [20]:
y = features[['radiant_win']]
y= y.fillna(0)

### Обучение классификатора методом градиентного бустинга

#### Фиксация генератора кроссвалидации

In [21]:
from sklearn.cross_validation import KFold
cv = KFold(object_count, n_folds=5, shuffle=True)

#### Обучение классификаторов с различным числом деревьев

In [22]:
estimator_counts = [5, 10, 20, 30, 40, 50, 60, 70]

In [23]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import time
import datetime

def calc_score(n, train_index, test_index):
    x_train = x.values[train_index]
    x_test = x.values[test_index]
    y_train = y.values[train_index]
    y_test = y.values[test_index]
    
    clf = GradientBoostingClassifier(n_estimators=n)
    
    clf.fit(x_train, np.ravel(y_train))    
    predict = clf.predict_proba(x_test)[:, 1]   
    return roc_auc_score(y_test, predict)



for n_estimators in estimator_counts:
    
    start_time = datetime.datetime.now()
    
    scores = [ calc_score(n_estimators, train_index, test_index) for train_index, test_index in cv ]
    score = np.array(scores).mean()
    
    end_time = datetime.datetime.now()
    
    total_time = end_time - start_time
    
    print("Число деревьев: %s, roc_auc_score: %s, потраченное время: %s" % (n_estimators, score, total_time))
    

Число деревьев: 5, roc_auc_score: 0.636389025824, потраченное время: 0:00:20.502776
Число деревьев: 10, roc_auc_score: 0.664622511743, потраченное время: 0:00:41.776878
Число деревьев: 20, roc_auc_score: 0.682562900874, потраченное время: 0:01:19.181556


KeyboardInterrupt: 

### Обучение классификатора методом логистической регрессии

#### Нормализация признаков

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

xs = scaler.fit_transform(x)

#### Наивное применение логистической регрессии

In [25]:
from sklearn.linear_model import LogisticRegression
regularizations = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1]

def calc_score_log(xs, c, train_index, test_index):
    x_train = xs[train_index]
    x_test = xs[test_index]
    y_train = y.values[train_index]
    y_test = y.values[test_index]
    
    clf = LogisticRegression(C=c)
    
    clf.fit(x_train, np.ravel(y_train))    
    predict = clf.predict_proba(x_test)[:, 1] 
    return roc_auc_score(y_test, predict)

def estimate_quality(xs):
    
    for c in regularizations:
    
        start_time = datetime.datetime.now()
    
        scores = [ calc_score_log(xs, c, train_index, test_index) for train_index, test_index in cv ]
        score = np.array(scores).mean()
    
        end_time = datetime.datetime.now()
    
        total_time = end_time - start_time
    
        print("Параметр регуляризации: %s, roc_auc_score: %s, потраченное время: %s" % (c, score, total_time))
    


In [26]:
estimate_quality(xs)

Параметр регуляризации: 1e-06, roc_auc_score: 0.688133316801, потраченное время: 0:00:01.525958
Параметр регуляризации: 1e-05, roc_auc_score: 0.695066604472, потраченное время: 0:00:01.917500
Параметр регуляризации: 0.0001, roc_auc_score: 0.711217407224, потраченное время: 0:00:03.330450
Параметр регуляризации: 0.001, roc_auc_score: 0.716149816261, потраченное время: 0:00:06.444137
Параметр регуляризации: 0.01, roc_auc_score: 0.716288894731, потраченное время: 0:00:08.258238
Параметр регуляризации: 0.1, roc_auc_score: 0.716256958017, потраченное время: 0:00:08.737845
Параметр регуляризации: 1, roc_auc_score: 0.716252727716, потраченное время: 0:00:08.725670


#### Логистическая регрессия с отброшенными категориальными признаками

In [27]:
 category_features = ['lobby_type',
                      'r1_hero',
                      'r2_hero',
                      'r3_hero',
                      'r4_hero',
                      'r5_hero',
                      'd1_hero',
                      'd2_hero',
                      'd3_hero',
                      'd4_hero',
                      'd5_hero']

In [28]:
x_nocat = x.drop(category_features, 1)
xs_nocat = scaler.fit_transform(x_nocat)

In [29]:
estimate_quality(xs_nocat)

Параметр регуляризации: 1e-06, roc_auc_score: 0.688088008126, потраченное время: 0:00:01.389024
Параметр регуляризации: 1e-05, roc_auc_score: 0.695001195997, потраченное время: 0:00:01.787333
Параметр регуляризации: 0.0001, roc_auc_score: 0.711184363166, потраченное время: 0:00:02.912995
Параметр регуляризации: 0.001, roc_auc_score: 0.716151285654, потраченное время: 0:00:05.786601
Параметр регуляризации: 0.01, roc_auc_score: 0.716295246548, потраченное время: 0:00:07.140980
Параметр регуляризации: 0.1, roc_auc_score: 0.716262290445, потраченное время: 0:00:08.075905
Параметр регуляризации: 1, roc_auc_score: 0.716257844964, потраченное время: 0:00:08.394962


#### Поиск числа различных значений для категориальных параметров

In [30]:
feature_value_counts = {feature_name: x[feature_name].value_counts().count() for feature_name in category_features}
feature_value_counts

{'d1_hero': 108,
 'd2_hero': 108,
 'd3_hero': 108,
 'd4_hero': 108,
 'd5_hero': 108,
 'lobby_type': 3,
 'r1_hero': 108,
 'r2_hero': 108,
 'r3_hero': 108,
 'r4_hero': 108,
 'r5_hero': 108}

In [31]:
feature_value_max = {feature_name: x[feature_name].max() for feature_name in category_features}
feature_value_max

{'d1_hero': 112,
 'd2_hero': 112,
 'd3_hero': 112,
 'd4_hero': 112,
 'd5_hero': 112,
 'lobby_type': 7,
 'r1_hero': 112,
 'r2_hero': 112,
 'r3_hero': 112,
 'r4_hero': 112,
 'r5_hero': 112}

#### Формирование "мешка слов" для категориальных атрибутов

In [32]:
N = feature_value_max['d1_hero']

In [33]:
x_pick = np.zeros((x.shape[0], N))

for i, match_id in enumerate(x.index):
    for p in range(5):
        x_pick[i, x.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        x_pick[i, x.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [34]:
xs_with_pick = np.concatenate((xs_nocat, x_pick), axis=1)

#### Логистическая регрессия с учтенными категориальными признаками выбора героев

In [35]:
estimate_quality(xs_with_pick)

Параметр регуляризации: 1e-06, roc_auc_score: 0.690491361766, потраченное время: 0:00:01.572593
Параметр регуляризации: 1e-05, roc_auc_score: 0.699115031175, потраченное время: 0:00:01.926339
Параметр регуляризации: 0.0001, roc_auc_score: 0.724895942549, потраченное время: 0:00:03.253581
Параметр регуляризации: 0.001, roc_auc_score: 0.746068923431, потраченное время: 0:00:07.132081
Параметр регуляризации: 0.01, roc_auc_score: 0.751465856173, потраченное время: 0:00:12.708235
Параметр регуляризации: 0.1, roc_auc_score: 0.751685712407, потраченное время: 0:00:19.595579
Параметр регуляризации: 1, roc_auc_score: 0.751671174501, потраченное время: 0:00:20.469258


### Подсчет предсказаний на тестовой выборке

In [148]:
clf = LogisticRegression(C=0.01)
clf.fit(xs_with_pick, np.ravel(y))

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [182]:
x_test = pd.read_csv("data/features_test.csv", index_col='match_id')

In [183]:
x_test = x_test.fillna(0)

In [184]:
xs_test = scaler.fit_transform(x_test)

In [185]:
x_test_pick = np.zeros((x_test.shape[0], N))

for i, match_id in enumerate(x_test.index):
    for p in range(5):
        x_test_pick[i, x_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        x_test_pick[i, x_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [186]:
x_test_nocat = x_test.drop(category_features, 1)
xs_test_nocat = scaler.fit_transform(x_test_nocat)

In [187]:
xs_test_with_pick = np.concatenate((xs_test_nocat, x_test_pick), axis=1)

In [199]:
test_predicts = clf.predict_proba(xs_test_with_pick)[:, 1]

In [201]:
result = np.zeros((x_test.shape[0], 2))

for i, match_id in enumerate(x_test.index):
    result[i, 0] = match_id
    result[i, 1] = test_predicts[i]
    

In [211]:
result_df = pd.DataFrame(result, columns=['match_id','radiant_win'])
result_df[['match_id']] = result_df[['match_id']].astype(int)

In [212]:
result_df.to_csv("test_for_kaggle.csv", index=False)

In [219]:
test_predicts.max()

0.9959831732360146

In [220]:
test_predicts.min()

0.0084426161409553949